<a href="https://colab.research.google.com/github/KimGriezmann/DeepLearning_BeTheLegend/blob/master/Bethelegend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tqdm
import os

In [3]:
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
print(os.getcwd())

/content


In [8]:
data = pd.read_excel("/content/날짜별 타자 데이터 통합.xlsx")
hitter = pd.read_excel("/content/90명 타자 명단.xlsx")
vs_avg_data = pd.read_excel("/content/팀 상대 타율 정렬.xlsx")

In [9]:
data = pd.DataFrame(data)
hitter = pd.DataFrame(hitter)
vs_avg_data = pd.DataFrame(vs_avg_data)

In [10]:
print(data.head(), len(data))

     day name  vs  타순  P  ab  hit  vs_cu_ab  vs_cu_hit  result    avg    slg  \
0  44289  강민호  키움   6  C   3    0         4          0       0  0.000  0.000   
1  44290  강민호  키움   7  C   4    2         5          4       1  0.286  0.286   
2  44292  강민호  두산   9  C   1    0         4          0       0  0.250  0.250   
3  44293  강민호  두산   6  C   4    1         3          1       1  0.250  0.333   
4  44294  강민호  두산   7  C   3    2         0          0       1  0.333  0.667   

   recent_5  recent_10  success_recent_10   LT     OB   LG  SSG   KT  \
0  0.000000      0.000               0.00  0.0  0.000  0.0  0.0  0.0   
1  0.000000      0.000               0.00  0.0  0.000  0.0  0.0  0.0   
2  0.285714      0.286               0.50  0.0  0.000  0.0  0.0  0.0   
3  0.250000      0.250               0.33  0.0  0.200  0.0  0.0  0.0   
4  0.250000      0.250               0.50  0.0  0.375  0.0  0.0  0.0   

         KW  KIA   HH   SS   NC  
0  0.000000  0.0  0.0  0.0  0.0  
1  0.285714  0.0  

In [11]:
print(hitter.head(), len(hitter))

  Team batter
0   롯데    전준우
1   롯데    손아섭
2   롯데     정훈
3   롯데    마차도
4   롯데    한동희 90


In [12]:
data.loc[:,['day','name','ab','hit']].dtypes

day      int64
name    object
ab       int64
hit      int64
dtype: object

In [13]:
data.loc[:,['day','name','ab','hit']]

,day,name,ab,hit
0,44289,강민호,3,0
1,44290,강민호,4,2
2,44292,강민호,1,0
3,44293,강민호,4,1
4,44294,강민호,3,2
...,...,...,...,...
5291,44371,힐리,1,0
5292,44372,힐리,2,0
5293,44373,힐리,4,2
5294,44374,힐리,3,1


In [13]:
#
# 선수마다 최근 5일의 타율을 구해야함. 
# 현재 day에서 -5부터 -1까지의 sum(hit)/sum(ab) 값을 구하면 된다

# hitter_data : 90명 타자 data
# record_data : 날짜별 타자 기록
# recent_day : 기본(최근 5일), 설정 가능
def make_recent_days(hitter_data, record_data, recent_day=5):

  # 최근 5일 경기 타율 저장할 공간 생성
  recent_5days = np.zeros(len(record_data)) # 최근 5일 경기 타율
  recent_5days_hitter = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  recent_5days_day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h] # 특정 타자 한명에 대해서만 데이터 추출
    for r in hdata['day']: # 추출한 데이터에서 한 행씩 접근
      recent_5days_hitter[ps] = h # 현재 타자
      recent_5days_day[ps] = r  # 현재 기준 날짜
      hrdata = hdata.loc[hdata['day']>=r-recent_day].loc[hdata['day']<=r-1] # 접근한 행의 날짜를 기준으로 5일 경기 
      if len(hrdata)==0: # 최근 5일 경기 없으면 넘어가고
        ps+=1
        continue
      else : # 있으면
        sum_ab = sum(hrdata['ab'])
        if sum_ab==0: # 경기 기록은 존재하지만 타수(ab)가 0
          ps+=1
          continue
        sum_hit = sum(hrdata['hit'])
        recent_5days[ps] = sum_hit/sum_ab
        ps+=1

  result_Dataframe = pd.DataFrame()
  result_Dataframe['recent_5days_hitter'] = recent_5days_hitter
  result_Dataframe['recent_5days_day'] = recent_5days_day
  result_Dataframe['recent_5days'] = recent_5days
  result_Dataframe

  return result_Dataframe

In [14]:
vs_avg_data.head()

,day,name,vs,타순,P,ab,hit,vs_cu_ab,vs_cu_hit,result,avg,slg,recent_5,recent_10,success_recent_10
0,44299,강민호,한화,5,C,4,2,0,0,1,0.367,0.533,0.388889,0.346,0.75
1,44300,강민호,한화,6,C,3,0,4,2,0,0.333,0.485,0.444444,0.367,0.78
2,44301,강민호,한화,6,C,4,2,7,2,1,0.351,0.486,0.333333,0.333,0.70
3,44321,강민호,한화,4,C,4,1,11,4,1,0.386,0.614,0.352941,0.421,0.80
4,44322,강민호,한화,8,PH,1,0,15,5,0,0.382,0.607,0.294118,0.405,0.80


In [62]:
#상대 타율 구하는 함수
#데이터 파일에서 vs 내림차순 후, 누적 타수 / 누적 안타 계산이 끝난 상태여야함
def make_vs_avg(hitter_data, record_data):
  vs_avg = np.zeros(len(record_data))
  name = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  vs = np.empty(len(record_data), dtype=np.chararray)
  vs_ab = np.zeros(len(record_data))
  vs_hit = np.zeros(len(record_data))

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h] # 특정 타자 한명에 대해서만 데이터 추출
    for r, v,ab, hit in zip(hdata['day'], hdata['vs'], hdata['vs_cu_ab'], hdata['vs_cu_hit']): # 추출한 데이터에서 한 행씩 접근
      day[ps] = r  # 현재 기준 날짜
      name[ps] = h # 현재 타자
      vs[ps] = v
      vs_ab[ps] = ab
      vs_hit[ps] = hit
      try:
        vs_avg[ps] = hit / ab
      except ZeroDivisionError:
        vs_avg[ps] = 0
      ps+=1
  
  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = day
  result_Dataframe['name'] = name
  result_Dataframe['vs_team'] = vs
  result_Dataframe['vs_ab'] = vs_ab
  result_Dataframe['vs_hit'] = vs_hit
  result_Dataframe['vs_avg'] = vs_avg
  result_Dataframe

  return result_Dataframe



In [63]:
vs_avg = make_vs_avg(hitter, vs_avg_data)

100%|██████████| 90/90 [00:00<00:00, 896.72it/s]


In [64]:
vs_avg.loc[vs_avg['name']=='전준우']

,day,name,vs_team,vs_ab,vs_hit,vs_avg
0,44316,전준우,한화,0.0,0.0,0.000000
1,44317,전준우,한화,4.0,1.0,0.250000
2,44318,전준우,한화,8.0,1.0,0.125000
3,44334,전준우,한화,11.0,1.0,0.090909
4,44335,전준우,한화,16.0,3.0,0.187500
5,44362,전준우,한화,18.0,4.0,0.222222
6,44363,전준우,한화,22.0,4.0,0.181818
7,44363,전준우,한화,25.0,5.0,0.200000
8,44364,전준우,한화,28.0,5.0,0.178571
9,44295,전준우,키움,0.0,0.0,0.000000


In [65]:
recent_days = make_recent_days(hitter, data)

100%|██████████| 90/90 [00:08<00:00, 10.78it/s]


In [50]:
recent_days

,recent_5days_hitter,recent_5days_day,recent_5days
0,전준우,44290,0.000000
1,전준우,44292,0.500000
2,전준우,44293,0.714286
3,전준우,44294,0.636364
4,전준우,44295,0.533333
...,...,...,...
5291,김민하,44367,0.222222
5292,김민하,44369,0.000000
5293,김민하,44370,0.000000
5294,김민하,44371,0.100000


In [66]:
merge_data1 = data.merge(recent_days, left_on = ['day', 'name'], right_on = ['recent_5days_day', 'recent_5days_hitter'])

In [67]:
merge_data2 = merge_data1.merge(vs_avg, left_on = ['day', 'name'], right_on = ['day', 'name'])

In [68]:
our_data = merge_data2.loc[:,['day','name', 'vs', 'ab','hit', 'avg', 'slg', 'vs_ab', 'vs_hit', 'vs_avg', 'recent_5','recent_10','success_recent_10','recent_5days', 'result']]

In [70]:
our_data.loc[our_data['name']=='전준우']

,day,name,vs,ab,hit,avg,slg,vs_ab,vs_hit,vs_avg,recent_5,recent_10,success_recent_10,recent_5days,result
4305,44290,전준우,SSG,4,2,0.500,0.750,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,1
4306,44292,전준우,NC,3,3,0.714,0.857,0.0,0.0,0.000000,0.500000,0.500,1.0,0.500000,1
4307,44293,전준우,NC,4,2,0.636,0.818,3.0,3.0,1.000000,0.714286,0.714,1.0,0.714286,1
4308,44294,전준우,NC,4,1,0.533,0.667,7.0,5.0,0.714286,0.636364,0.636,1.0,0.636364,1
4309,44295,전준우,키움,4,1,0.474,0.579,0.0,0.0,0.000000,0.533333,0.533,1.0,0.533333,1
4310,44296,전준우,키움,4,3,0.522,0.609,4.0,1.0,0.250000,0.473684,0.474,1.0,0.466667,1
4311,44297,전준우,키움,4,1,0.481,0.556,8.0,4.0,0.500000,0.526316,0.522,1.0,0.526316,1
4312,44299,전준우,KIA,5,2,0.469,0.531,0.0,0.0,0.000000,0.400000,0.481,1.0,0.375000,1
4313,44300,전준우,KIA,5,1,0.432,0.486,5.0,2.0,0.400000,0.380952,0.469,1.0,0.411765,1
4314,44301,전준우,KIA,3,2,0.450,0.500,10.0,3.0,0.300000,0.363636,0.432,1.0,0.388889,1
